In [ ]:
import wfdb
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as np
import pywt
import math

https://www.sciencedirect.com/science/article/pii/S0957417411006075

In [ ]:
def reconstruction_plot(yyy,**kwargs):
    """Plot signal vector on x [0,1] independently of amount of values it contains."""
    plt.plot(np.linspace(0, len(ecg), len(yyy)), yyy, **kwargs)

In [ ]:
def wrcoef(X, coef_type, coeffs, wavename, level):
    N = np.array(X).size
    a, ds = coeffs[0], list(reversed(coeffs[1:]))

    if coef_type =='a':
        return pywt.upcoef('a', a, wavename, level=level)[:N]
    elif coef_type == 'd':
        return pywt.upcoef('d', ds[level-1], wavename, level=level)[:N]
    else:
        raise ValueError("Invalid coefficient type: {}".format(coef_type))

In [ ]:
fileName=['a01','a02','a03','a04','a05','a06','a07','a08','a09','a10','a11','a12','a13','a14','a15','a16','a17','a18','a19','a20',
         'b01','b02','b03','b04','b05','c01','c02','c03','c04','c05','c06','c07','c08','c09','c10',
         'x01','x02','x03','x04','x05','x06','x07','x08','x09','x10','x11','x12','x13','x14','x15','x16','x17','x18','x19','x20']
mypath='C:/Users/sara/Documents/5anno/TESI/DataBase_PhysioNet/www.physionet.org/physiobank/database/apnea-ecg/'


In [ ]:
recordname=mypath+fileName[0]
record = wfdb.rdsamp(recordname)
dfECG=pd.DataFrame()
ecg=record[0]
Fs_ecg=record[1]['fs']
intervallo=len(ecg)/(len(ecg)*Fs_ecg)
time_ecg=[]
for i in range(0,len(ecg)):
    time_ecg.append(intervallo*i)
dfECG['time']=time_ecg
dfECG['ecg']=ecg


In [ ]:
wavelet = pywt.Wavelet('db6')
print(wavelet)
#[C,l]=pywt.dwt(ecg, wavelet)

In [ ]:
from pywt import wavedec
ecg=np.array(dfECG['ecg'])
coeffs = wavedec(ecg, 'db6', level=7)


In [ ]:
plt.figure(figsize=(14,10))
plt.axis([0,500,-3,3])
reconstruction_plot(pywt.waverec(coeffs, wavelet)) # complete reconstruction
reconstruction_plot(pywt.waverec(coeffs[:-1] + [None] * 1, wavelet)) # leaving out detail coefficients up to lvl 15
reconstruction_plot(pywt.waverec(coeffs[:-2] + [None] * 2, wavelet)) # leaving out detail coefficients up to lvl 2
reconstruction_plot(pywt.waverec(coeffs[:-3] + [None] * 3, wavelet)) # leaving out detail coefficients up to lvl 3
reconstruction_plot(pywt.waverec(coeffs[:-4] + [None] * 4, wavelet)) # leaving out detail coefficients up to lvl 4
reconstruction_plot(pywt.waverec(coeffs[:-5] + [None] * 5, wavelet)) # leaving out detail coefficients up to lvl 5
reconstruction_plot(pywt.waverec(coeffs[:-6] + [None] * 6, wavelet)) # leaving out detail coefficients up to lvl 6
reconstruction_plot(pywt.waverec(coeffs[:-7] + [None] * 7, wavelet)) # only approx
plt.legend(['Full reconstruction', 'Reconstruction using lv1 to lv7','Reconstruction using lv2 to lv7','Reconstruction using lv3 to lv7'
           ,'Reconstruction using lv4 to lv7','Reconstruction using lv5 to lv7','Reconstruction using lv6 to lv7','approx'])

In [ ]:
level = 7
A7 = wrcoef(ecg, 'a', coeffs, 'db6', level)
D7 = wrcoef(ecg, 'd', coeffs, 'db6', level)
D6 = wrcoef(ecg, 'd', coeffs, 'db1', 6)
D5 = wrcoef(ecg, 'd', coeffs, 'db6', 5)
D4 = wrcoef(ecg, 'd', coeffs, 'db6', 4)
D3 = wrcoef(ecg, 'd', coeffs, 'db6', 3)
D2 = wrcoef(ecg, 'd', coeffs, 'db6', 2)
D1 = wrcoef(ecg, 'd', coeffs, 'db6', 1)

In [ ]:
rec=A7+D7+D6+D5+D4+D3+D2+D1

In [ ]:
f, axarr = plt.subplots(9, sharex=True,figsize=(15,8))
axarr[0].axis([0,5000,-3,3])
axarr[0].plot(D1)
axarr[1].plot(D2)
axarr[2].plot(D3)
axarr[3].plot(D4)
axarr[4].plot(D5)
axarr[5].plot(D6)
axarr[6].plot(D7)
axarr[7].plot(A7)
axarr[8].plot(ecg)

### Tolgo il movimento dal segnale ECG

In [ ]:
ecg_rec=[]
for i in range(0,len(ecg)):
    ecg_rec.append(ecg[i]-A7[i])

In [ ]:
#tengo i valori assoluti di D2
D2_abs=[math.sqrt(i**2) for i in D2]

In [ ]:
ms=0
for i in range(0,len(D2)):
    ms=ms+D2_abs[i]**2
ms=ms/len(D2)
rms=math.sqrt(ms)

In [ ]:
D2_abs_th=[]
for i in range(0,len(D2_abs)):
    if(D2_abs[i]<rms*2):
        D2_abs_th.append(0)
    else:
        D2_abs_th.append(D2_abs)

In [ ]:
s

In [ ]:
plt.plot(D2_abs_th)

In [ ]:
from scipy.signal import find_peaks

In [ ]:
peaks, _ = find_peaks(D2_abs_th, distance=150)
np.diff(peaks)
array([186, 180, 177, 171, 177, 169, 167, 164, 158, 162, 172])
plt.plot(D2_abs_th)
plt.plot(peaks, D2_abs_th[peaks], "D2")